<a href="https://colab.research.google.com/github/hmblackwood/Medical_Abstract_Project/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🩺 Medical Abstract Project 🩺

# Summary:
Replicating the deep learning model behind the white paper PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts.
Created a RNN, multi-__ clasifier model with ___.
Libraries and Project used: PyTorch, SkimLit

##1. Problem
The number of Randomized Controlled Trials (RCTs) increases every year. Reading completely through even a small portion of them might not be the best use of a medical professional's time. I will create a model that will summarize the abstract, ___, ___ and ___ to help them skim through large bodies of information.

##2. Data
The dataset I'll be working with is provided in PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts at https://arxiv.org/abs/1710.06071

##3. Machine Learning Model
I will be replicating the deep learning model behind the paper, PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts. This paper presented a new dataset called PubMed 200k RCT which consists of ~200,000 labelled Randomized Controlled Trial (RCT) abstracts.
https://arxiv.org/abs/1710.06071

In this paper, they state that they use the machine learning model described in Neural Networks for Joint Sentence Classification in Medical Paper Abstracts. I will be replicating this model.
https://arxiv.org/abs/1612.05251

From the white paper:
"The fourth baseline (bi-ANN) is an ANN consisting of three components: a token embedding
layer (bi-LSTM), a sentence label prediction layer
(bi-LSTM), and a label sequence optimization
layer (CRF). The architecture is described in (Dernoncourt et al., 2016) and has been demonstrated
to yield state-of-the-art results for sequential sentence classification."


##4. Evaluation

##5. Features

I will perform the following steps:
1. Download the text dataset (PubMed 200K RCT)
2. Code a preprocessing function for my text data.
3. Set up multiple modeling experiments with different levels of token embeddings.
  - Make a baseline (TF-IDF classifier)
  - Deep models with various combinations of: token embeddings, character embeddings, pretrained embeddings, positional embeddings.
4. Build a multimodal model to take in different sources of data.
5. Find the most wrong prediction examples ---- purpose?
6. Make predictions on PubMed abstracts in the wild.


## Confirm Access to a GPU

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Download the Dataset
 Dataset can be found on the author's GitHub:
 https://github.com/Franck-Dernoncourt/pubmed-rct

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25 (from 1)
Receiving objects: 100% (39/39), 177.08 MiB | 31.49 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.
PubMed_200k_RCT				       PubMed_20k_RCT_numbers_replaced_with_at_sign
PubMed_200k_RCT_numbers_replaced_with_at_sign  README.md
PubMed_20k_RCT


In [3]:
# Look at what files are in the dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

dev.txt  test.txt  train.txt


✅ The dataset has a test and train set already. Note that the validation set is called "dev."

In [4]:
# I'll begin experiments using the 20k dataset of with the numbers replaced by the @ sign.
data_dir = "/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [5]:
# Check all of the filenames in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

# Preprocess Data
I will write a function to read in all the lines of the target text file.

In [8]:
# Create function to read the lines of a document
def get_lines(filename):
  """
  Reads the filename (text filename) and returns the lines of text as a list.

  Args:
    filename: target file path in the form of a string.

  Returns:
    A list of strings with one string per line.
  """
  with open(filename, "r") as f:
    return f.readlines()


In [9]:
# Read in the training lines
train_lines = get_lines(data_dir+"train.txt") # read the lines with the training file
train_lines[:27]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

✅ \t means tab in the data.
I need a function to separate labels from the text and make it easier for our model to take in.

In [10]:
len(train_lines)

210040